<a target="_blank" href="https://colab.research.google.com/github/trainocate-japan/Machine-Learning-and-Deep-Learning-Hands-on/blob/main/exercise/2_線形回帰/2-1-1_線形回帰による中古マンションの価格予測①.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 2-1-1_線形回帰による中古マンションの価格予測

このノートブックでは、機械学習の基本的な作業工程や機械学習でよく使うライブラリを試しながら、線形回帰で予測モデルを作成します。

予測を行うテーマは物件の価格を予測することです。<br>
神奈川県にある、JR東海道線の戸塚駅周辺物件情報を用意しました。2019年4月～2021年3月までの2年間で売買が成立した中古マンションの、住宅価格と価格以外の情報を確認して、価格以外の情報から住宅価格を予測してみましょう。

## ライブラリのインポート

In [ ]:
# データを処理するための基本的なライブラリ
# どれも途中で使用することが多いので、使うかどうかにかかわらずひとまずインポートしてしまってもよいでしょう
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# scikit-learnから機械学習モデルをインポート。今回は線形回帰モデル。
from sklearn.linear_model import LinearRegression

In [ ]:
'''
matplotlibに日本語を表示させたいときには以下のコマンドを実行して、japanize_matplotlibをインストールしておく。
Google Colaboratoryにプリインストールされていないので、インストール作業が必要。
'''
# !を先頭に入れるとBashコマンドを入力できる。japanize_matplotlibをインストールする。
!pip install japanize_matplotlib | tail -n 1
# japanize_matplotlibをインポート
import japanize_matplotlib

## データの準備

#### データを取り込む
- pandasのread_csvメソッドを使用して、totsuka_used.csvファイルを読み込みます
- read_csvメソッドは引数に指定したファイルをpandasのDataFrame型で返します

In [ ]:
# csvファイルを読み込みます
totsuka_used = pd.read_csv("totsuka_used.csv")

#### データを確認する

In [ ]:
# 読み込んだデータを確認します
# DataFrameに用意されているheadメソッドは先頭の5行分のデータを出力します。
totsuka_used.head()

データの全体像を確認します。

取り込んだデータが意図したとおりの型・形状になっているか、欠損値（歯抜けになっている値）がないか、最小・最大・平均値の値はどうなっているかなど、機械学習を行う前にこれから予測するデータについて理解を深めておきます。

In [ ]:
# totsuka_usedのデータ要約を確認
totsuka_used.info()

In [ ]:
# totsuka_usedの統計情報を確認
totsuka_used.describe()

#### 説明変数、目的変数を切り出す
- 説明変数はtotsuka_usedから「築年数」を選択し、変数名をtrain_xとして代入しましょう
- 目的変数はtotsuka_usedから「取引価格」を選択し、変数名をtrain_yとして代入しましょう

In [ ]:
# 列の指定に[]を二重に記載していますが、これは2次元の形式（DataFrame型）であることを明示するためです
train_x = totsuka_used[['築年数']]

In [ ]:
train_y = totsuka_used['取引価格']

## 機械学習モデルを定義する


In [ ]:
# 変数modelに線形回帰の機械学習モデルのインスタンスを格納する
model = LinearRegression()

## 機械学習モデルの学習
- 作成したモデルのインスタンスに対してfitメソッドを使います
- 引数として、用意した、説明変数、目的変数を順に指定します
- fitを実行すると与えたデータを用いて学習を行い、予測を行うことができるようになります
- データをNumPy配列ではなくDataFrameで渡していますが、自動で変換が行われます。

In [ ]:
model.fit(train_x, train_y)

## 機械学習モデルの評価
出来上がった線形回帰の方程式を確認してみます。

In [ ]:
# 傾きを求めるにはcoef_プロパティを使う
# y = ax + bのaの部分
model.coef_

In [ ]:
# 切片を求めるにはintercept_プロパティを使う
# y = ax + bのbの部分
model.intercept_

出来上がった方程式は「y = -725666x + 43175674」です。<br>
築年数が0年の時4300万円と予測し、築年数が一年増えるごとに70万円ほど価格が減少するような計算式となりました。

#### 予測をする
- 予測をするにはpredictメソッドを使います
- predictメソッドの引数に説明変数を指定します
- predictメソッドの引数はnumpy配列です。複数まとめてデータを渡すことができます。一件だけ予測したい場合にも配列で渡します。（ここでもDataFrameが自動で変換されます）
- 同じくpredictメソッドの戻り値はnumpy配列です。複数まとめて結果を受け取ることができます。

In [ ]:
# train_xの値をすべて渡して予測し、そのうち最初の5件だけを表示する
model.predict(train_x)[:5]

In [ ]:
# 正解を確認してみる
train_y.head()

In [ ]:
# 入力されたtrain_xの値も確認してみる
train_x.head(5)

#### 精度を確認する
評価は線形回帰のモデルに用意されているscoreメソッドで行います。<br>
回帰の場合の精度は決定係数R2で計算されます。

In [ ]:
# 精度を確認してみる
model.score(train_x, train_y)

このノートブックは以上です。
2-1-2のノートブックに続きます。